In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from uncertainties import ufloat
from uncertainties import unumpy
from scipy import odr
import pandas as pd

# Resonance using Lock-in Amplifier

In [ ]:
# amplitude fit
def res_LIA_Amplitude(beta, omega):
    omega_0, gamma, A, C = beta
    return (A/np.sqrt((omega_0**2 - omega**2)**2 + omega**2 * gamma**2)) + C

def fit_resonance_Amplitude(omega_u, R_V_u, p0, title="LIA"):
    model = odr.Model(res_LIA_Amplitude)
    data = odr.RealData(unumpy.nominal_values(omega_u), unumpy.nominal_values(R_V_u), sx=unumpy.std_devs(omega_u), sy=unumpy.std_devs(R_V_u))
    odr_run = odr.ODR(data, model, beta0=p0).run()

    omega0_fit, gamma_fit, A_fit, C_fit  = odr_run.beta
    omega0_err, gamma_err, A_err, C_err = odr_run.sd_beta

    omega_0 = ufloat(omega0_fit, omega0_err)
    gamma = ufloat(gamma_fit, gamma_err)
    A = ufloat(A_fit, A_err)
    C = ufloat(C_fit, C_err)

    # r2-score
    y_hat = res_LIA_Amplitude(odr_run.beta, unumpy.nominal_values(omega_u))
    ss_res = np.sum((unumpy.nominal_values(R_V_u) - y_hat)**2)
    ss_tot = np.sum((unumpy.nominal_values(R_V_u) - np.mean(unumpy.nominal_values(R_V_u)))**2)
    r_squared = 1.0 - ss_res/ss_tot

    # convert to Frequency
    f_0 = omega_0/(2*np.pi)

    # quality factor
    Q = omega_0/gamma

    print(f"f0 = {f_0:.2f} Hz")
    print(f"Gamma = {gamma:.2f} rad/s")
    print(f"Q = {Q:.2f}")

    # plot
    f_min, f_max = np.min(unumpy.nominal_values(omega_u)/(2*np.pi)), np.max(unumpy.nominal_values(omega_u)/(2*np.pi))
    f_pad = 0.05 * (f_max - f_min)
    f_fit = np.linspace(f_min - f_pad, f_max + f_pad, 1000)
    omega_fit = 2 * np.pi * f_fit

    R_fit  = res_LIA_Amplitude([omega0_fit, gamma_fit, A_fit, C_fit], omega_fit)

    plt.figure(figsize=(7.5, 5))
    plt.errorbar(unumpy.nominal_values(omega_u)/(2*np.pi), unumpy.nominal_values(R_V_u), xerr=unumpy.std_devs(omega_u)/(2*np.pi), yerr=unumpy.std_devs(R_V_u), fmt='o', capsize=3, label='Data')
    plt.plot(f_fit, R_fit, '-', label=rf"{title} fit")
    plt.xlabel('Frequency $f$ [Hz]')
    plt.ylabel('Voltage [V]')
    plt.title(f'Resonance with {title} (R² = {r_squared:.5f})')

    # styling for legend and text box done by ChatGPT
    # Draw legend first (so it exists and we can read its style)
    legend = plt.legend(loc='upper right', fontsize=9, frameon=True, alignment='left')
    plt.tight_layout()

    # ---- Get legend box properties ----
    legend_box = legend.get_frame()
    legend_style = dict(
        boxstyle=f"{legend_box.get_boxstyle().__class__.__name__.lower()} ,pad=0.35",
        facecolor=legend_box.get_facecolor(),
        edgecolor=legend_box.get_edgecolor(),
        linewidth=legend_box.get_linewidth()
    )

    # ---- Textbox content (ufloat compatible) ----
    text_box = '\n'.join((
        rf'$f_0 = ({f_0.n:.1f} \pm {f_0.s:.1f}) \,\mathrm{{Hz}}$',
        rf'$\Gamma = ({gamma.n:.0f} \pm {gamma.s:.0f})\,\mathrm{{rad/s}}$',
        rf'$A = ({A.n/1000:.0f} \pm {A.s/1000:.0f})\cdot 10^3 \, \mathrm{{m/s^2}}$',
    ))

    # ---- Add matching text box ----
    plt.text(
        0.765, # x
        0.72, # y
        text_box,
        transform=plt.gca().transAxes,
        fontsize=9,
        verticalalignment='bottom',
        bbox=legend_style,
    )

    plt.savefig(f'resonance_{title}_amplitude.png', dpi=600)
    plt.show()

In [ ]:
# phase fit
def res_LIA_Phase(beta, omega):
    omega_0, gamma, phi = beta
    return np.arctan2(omega * gamma , omega_0**2 - omega**2)

def fit_resonance_phase(omega_u, Phase_rad_u, p0, title="LIA"):
    model = odr.Model(res_LIA_Phase)
    data = odr.RealData(unumpy.nominal_values(omega_u), unumpy.nominal_values(Phase_rad_u), sx=unumpy.std_devs(omega_u), sy=unumpy.std_devs(Phase_rad_u))
    odr_run = odr.ODR(data, model, beta0=p0).run()

    omega0_fit, gamma_fit, phi_fit  = odr_run.beta
    omega0_err, gamma_err, phi_err_fit = odr_run.sd_beta

    omega_0 = ufloat(omega0_fit, omega0_err)
    gamma = ufloat(gamma_fit, gamma_err)
    phi = ufloat(phi_fit, phi_err_fit)

    # r2-score
    y_hat = res_LIA_Phase(odr_run.beta, unumpy.nominal_values(omega_u))
    ss_res = np.sum((unumpy.nominal_values(Phase_rad_u) - y_hat)**2)
    ss_tot = np.sum((unumpy.nominal_values(Phase_rad_u) - np.mean(unumpy.nominal_values(Phase_rad_u)))**2)
    r_squared = 1.0 - ss_res/ss_tot

    # convert to Frequency
    f_0 = omega_0/(2*np.pi)

    print(f"f0 = {f_0:.2f} Hz")
    print(f"Gamma = {gamma:.2f} rad/s")

    # plot
    f_min, f_max = np.min(unumpy.nominal_values(omega_u)/(2*np.pi)), np.max(unumpy.nominal_values(omega_u)/(2*np.pi))
    f_pad = 0.05 * (f_max - f_min)
    f_fit = np.linspace(f_min - f_pad, f_max + f_pad, 1000)
    omega_fit = 2 * np.pi * f_fit

    Phase_fit  = res_LIA_Phase([omega0_fit, gamma_fit, phi_fit], omega_fit)

    plt.figure(figsize=(7.5, 5))
    plt.errorbar(unumpy.nominal_values(omega_u)/(2*np.pi), unumpy.nominal_values(Phase_rad_u), xerr=unumpy.std_devs(omega_u)/(2*np.pi), yerr=unumpy.std_devs(Phase_rad_u), fmt='o', capsize=3, label='Data')
    plt.plot(f_fit, Phase_fit, '-', label=rf"{title} fit")
    plt.xlabel('Frequency $f$ [Hz]')
    plt.ylabel('Phase [rad]')
    plt.title(f'Resonance with {title} (R² = {r_squared:.5f})')

    # styling for legend and text box done by ChatGPT
    # Draw legend first (so it exists and we can read its style)
    legend = plt.legend(loc='upper right', fontsize=9, frameon=True, alignment='left')
    plt.tight_layout()

    # ---- Get legend box properties ----
    legend_box = legend.get_frame()
    legend_style = dict(
        boxstyle=f"{legend_box.get_boxstyle().__class__.__name__.lower()} ,pad=0.35",
        facecolor=legend_box.get_facecolor(),
        edgecolor=legend_box.get_edgecolor(),
        linewidth=legend_box.get_linewidth()
    )

    # ---- Textbox content (ufloat compatible) ----
    text_box = '\n'.join((
        rf'$f_0 = {f_0.n:.1f} \pm {f_0.s:.1f}\,\mathrm{{Hz}}$',
        rf'$\Gamma = {gamma.n:.0f} \pm {gamma.s:.0f}\,\mathrm{{rad/s}}$',
    ))

    # ---- Add matching text box ----
    plt.text(
        0.81, # x
        0.77, # y
        text_box,
        transform=plt.gca().transAxes,
        fontsize=9,
        verticalalignment='bottom',
        bbox=legend_style,
    )

    plt.savefig(f'resonance_{title}_phase.png', dpi=600)
    plt.show()

In [ ]:
# adjust frequency in Hz
f_Hz = np.array([700,750,800,815,820,826,850,900,950,1000,1050,1100,1200])
omega_u = unumpy.uarray(f_Hz, np.full(len(f_Hz), 0.5)) * (2 * np.pi)

# adjust amplitude in V
R_vals = np.array([0.18,0.32,0.78,1.48,1.92,2.41,1.12,0.38,0.22,0.15,0.12,0.11,0.09])
R_err  = np.array([0.01,0.01,0.03,0.04,0.05,0.07,0.04,0.02,0.02,0.01,0.01,0.01,0.01])
R_V_u = unumpy.uarray(R_vals, R_err)

# adjust phase in V
phi_vals = np.array([4.299,  5.588,  9.418, 11.959, 16.186, 24.618, 49.733, 628.319, -25.946, -17.519, -13.295, -7.848, -6.547])
phi_err  = np.array([0.04,0.03,0.04,0.04,0.06,0.07,0.04,0.04,0.01,0.02,0.02,0.06,0.01])
Phase_rad_u = (unumpy.uarray(phi_vals, phi_err)/ 1000 * (2 * np.pi))%(np.pi)

p0 = [2*np.pi*830, 30, 100000, 1]
fit_resonance_Amplitude(omega_u, R_V_u, p0)

p0 = [2*np.pi*830, 107, 0]
fit_resonance_phase(omega_u, Phase_rad_u, p0)

# Resonance using frequency analyser

In [ ]:
# conversion from dB to V
def dBm_to_Vpp(data):
    return 10**(data/20)

# uncertainty approximation
def uncertainty_approx(x):
    s = str(x).strip()
    if "." in s:
        decimals = len(s.split(".")[1])
        return 0.5 * 10**(-decimals)
    else:
        return 0.5 # keine Nachkommastellen

In [ ]:
file_path = "sweep2_JL.csv"
df = pd.read_csv(file_path, comment="%", sep=",", header=None, names=["Time (Hz)", "Magnitude (dB)", "Phase (deg)"]) # dateien einlesen

cols = ["Magnitude (dB)", "Phase (deg)"]
df[cols] = df[cols].apply(lambda col: pd.to_numeric(col.astype(str).str.strip(), errors="coerce")) # get rid of spaces and convert anything to numbers

data = df.dropna(subset=["Magnitude (dB)", "Phase (deg)"])
f_Hz = np.array(data["Time (Hz)"].values)
A_dBm = np.array(data["Magnitude (dB)"].values)
Phase_deg = np.array(data["Phase (deg)"].values)

# conversions
A_V = dBm_to_Vpp(A_dBm) # convert dB to V
Phase_rad = (Phase_deg * np.pi / 180) % (2 * np.pi) # convert deg to rad

f_Hz_err = np.array([uncertainty_approx(i) for i in f_Hz])
A_V_err = np.array([uncertainty_approx(i) for i in A_V])
Phase_rad_err = np.array(([uncertainty_approx(i) for i in Phase_rad]))

omega_u = unumpy.uarray(f_Hz, f_Hz_err) * (2 * np.pi)
A_v_u = unumpy.uarray(A_V, A_V_err)
Phase_rad_u = unumpy.uarray(Phase_rad, Phase_rad_err)

p0 = [2*np.pi*836, 33, 50000, 1]

omega_0_p0 = 8.30e2    # Hz  (≈ 830.2)
gamma_p0   = 1.73e2     # rad/s  (≈ 173)
A_p0       = 2.75e7     # amplitude scale
C_p0       = -36.76     # dBm baseline

#p0 = [omega_0_p0, gamma_p0, A_p0, C_p0]

p0 = [2*np.pi*830, 29.36, 47622.5, 1]

fit_resonance_Amplitude(omega_u, A_v_u, p0, "FRA")

p0 = [2*np.pi*829.80, 229.9, -4.1]
fit_resonance_phase(omega_u, Phase_rad_u, p0, "FRA")

# Ringdown

In [ ]:
def ringdown(beta, t):
    A, gamma,omega_0, delta = beta
    omega = np.sqrt(omega_0**2-gamma**2)
    return A * np.exp(-gamma*t) * np.sin(omega*t + delta)

